In [ ]:
from selenium import webdriver
from time import sleep

In [ ]:
USERNAME=""
PASSWORD=""

In [ ]:
driver=webdriver.Chrome(r"C:\chromedriver-win64\chromedriver")
target_url="https://www.instagram.com/"
driver.get(target_url)
sleep(3)

In [ ]:
error_flg=False

In [ ]:
try:
    username_input=driver.find_element_by_xpath("//input[@aria-label='電話番号、ユーザーネーム、またはメールアドレス']")
    username_input.send_keys(USERNAME)
    sleep(1)

    password_input=driver.find_element_by_xpath("//input[@aria-label='パスワード']")
    password_input.send_keys(PASSWORD)
    sleep(1)

    login_button=driver.find_element_by_xpath("//button[@type='submit']")
    login_button.submit()
    sleep(1)
    
except Exception:
    error_flg=True
    print("ユーザー名、パスワード入力時にエラーが発生しました。")

In [ ]:
if error_flg is False:
    try:
        sleep(3)
        notnow_button1=driver.find_element_by_xpath("//div[@role='button' and text()='後で']")
        notnow_button1.click()
        sleep(3)
        notnow_button2=driver.find_element_by_xpath("//button[text()='後で']")
        notnow_button2.click()

    except Exception:
        pass

In [ ]:
target_username="paulnicklen"
if error_flg is False:
    try:
        target_profile_url=target_url+target_username
        driver.get(target_profile_url)
        # sleep(3)
        # notnow_button3=driver.find_element_by_element_by_xpath("//svg[@aria-label='閉じる']")
        # notnow_button3.click()
        sleep(1)

    except Exception:
        print("検索時にエラーが発生しました。")
        error_flg=True

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
if error_flg is False:
    try:
        post_count=driver.find_element_by_xpath("//div[contains(text(),'投稿')]/span/span").text
        # post_count=post_count.replace("投稿","").replace("件","")
        print("投稿件数："+post_count)
        post_count=int(post_count)
        if post_count>12:
            scroll_count=int(post_count/12)+1
            try:
                all_images=[]
                for i in range(scroll_count):
                    soup=BeautifulSoup(driver.page_source,"html.parser")
                    for image in soup.select("div._aagv img"):
                        all_images.append(image)
                    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
                    sleep(2)
                    if i>5:
                        break

                all_images=list(dict.fromkeys(all_images))
                for index, image in enumerate(all_images):
                    print("画像番号:"+str(index))
                    print("image['src']:"+image['src'],end="\n\n")
            except Exception:
                print("画面スクロール中にエラーが発生しました。")
                error_flg=True
    except Exception:
        print("投稿数が取得できませんでした。")
        error_flg=True

In [ ]:
import requests
import re 
import os
import shutil

path=r""

if error_flg is False:
    try:
        for index, image in enumerate(all_images):
            filename="image_"+str(index)+".jpg"
            image_path=os.path.join(path,filename)
            image_link=image["src"]
    
            url_ptn=re.compile(r"^(http|https)://")
            res=url_ptn.match(image_link)
            if res:
                response=requests.get(image_link,stream=True)
                with open(image_path,"wb") as file:
                    shutil.copyfileobj(response.raw, file)
    except Exception as e:
        print(e)
        print(str(index)+"番目の画像のダウンロード・保存時にエラーが発生しました。")
        print("画像へのリンク:"+image_link)